In [ ]:
import os
import numpy as np
import tweepy as tw
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import spacy
import re
import collections
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import time
import json
import sys
from datetime import datetime

In [ ]:
consumer_key = "xuSrX6LK9gfsWnJ1CTQblfL83"
consumer_secret = "lIyGqVBJygUHHQ4ZoHyAc8INULSr1B4WnuCajei13QINe7U3SH"
access_token = "549909021-ksoN2E4V4yAlhAPa5EUuItuiQhSXyIKFEvCHLqMv"
access_token_secret ="JplD62xq9UPnlQ0NFekGINJZHO1nEpfmw8Myiyl6PyaXS"

In [ ]:
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [ ]:
if __name__ == "__main__":
    # Available Locations
    available_loc = api.trends_available()
    # writing a JSON file that has the available trends around the world
    with open("available_locs_for_trend.json","w") as wp:
        wp.write(json.dumps(available_loc, indent=1))

    # Trends for Specific Country
    loc = 'Colombia'
    #loc = sys.argv[1]     # location as argument variable 
    g = geocoder.osm(loc) # getting object that has location's latitude and longitude

    closest_loc = api.trends_closest(g.lat, g.lng)
    #trends = api.trends_place(closest_loc[0]['woeid'])
    trends = api.trends_place(id = 23424787)
    # writing a JSON file that has the latest trends for that location
    #with open("twitter_{}_trend.json".format(loc),"w") as wp:
        #wp.write(json.dumps(trends[0]['trends'], indent=1))
    #Create DataFrame and write it to excel file
    pd_trend = pd.DataFrame(trends[0]['trends'])
    pd_trend['trend']=pd_trend.name
    #pd_trend.sort_values(by=['tweet_volume'],ascending = False).to_excel("twitter_{}_trend_{date:%Y-%m-%d_%H_%M_%S}.xlsx".format(loc,date=datetime.now()),index = False)
    #pd_trend=pd_trend.sort_values(by=['tweet_volume'],ascending = False,ignore_index =True)
    pd_trend['index'] = pd_trend.index+1
    pd_trend = pd_trend[['index','trend','tweet_volume']]
    pd_trend.to_excel("twitter_colombia_trend.xlsx",index = False)
    




In [ ]:
def get_woeid(place):
    '''Get woeid by location'''
    try:
        trends = api.trends_available()
        for val in trends:
            if (val['name'].lower() == place.lower()):
                return(val['woeid']) 
        print('Location Not Found')
    except Exception as e:
        print('Exception:',e)
        return(0)
      

In [ ]:
def get_trends(place):
    #woeid colombia = 23424787
    col_trends = api.trends_place(id =get_woeid(place))
    trends = []
    for trend in col_trends[0]['trends']:  
        trends.append((trend['name'], trend['tweet_volume']))
    pd_trends = pd.DataFrame(trends,columns = ['trend','tweet_volume'])
    #pd_trends[~pd_trends['trend'].isin(FilterList)]
    pd_trends.reset_index(inplace = True)
    pd_trends['index'] = pd_trends.index+1
    pd_trends = pd_trends[['index','trend','tweet_volume']]
    pd_trends.to_excel("twitter_colombia_trend.xlsx",index = False)

In [ ]:
get_trends('Colombia')

In [ ]:
data_trend = pd.read_excel("twitter_colombia_trend.xlsx")
data_trend

,index,trend,tweet_volume
0,1,#ParoNacional19M,117494.0
1,2,Marta Lucía Ramírez,NaN
2,3,#ReformaALaSalud,20334.0
3,4,Colmenares,NaN
4,5,Canciller,15649.0
5,6,Laura Moreno,NaN
6,7,Enzo Pérez,50346.0
7,8,Ley 100,NaN
8,9,#SaqueLargoWIN,NaN
9,10,#LaJuventudNecesita,NaN


In [ ]:
def scraptweets(search_words, date_since, numTweets, numRuns):
    
    # Define a for-loop to generate tweets at regular intervals
    # We cannot make large API call in one go. Hence, let's try T times
    
    # Define a pandas dataframe to store the date:
    db_tweets = pd.DataFrame(columns = ['username', 'acctdesc', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags']
                                )
    program_start = time.time()
    for i in range(0, numRuns):
        # We will time how long it takes to scrape tweets for each run:
        start_run = time.time()
        
        # Collect tweets using the Cursor object
        # .Cursor() returns an object that you can iterate or loop over to access the data collected.
        # Each item in the iterator has various attributes that you can access to get information about each tweet
        tweets = tw.Cursor(api.search, q=search_words, since=date_since,lang = 'es', tweet_mode='extended').items(numTweets)
# Store these tweets into a python list
        tweet_list = [tweet for tweet in tweets]
# Obtain the following info (methods to call them out):
        # user.screen_name - twitter handle
        # user.description - description of account
        # user.location - where is he tweeting from
        # user.friends_count - no. of other users that user is following (following)
        # user.followers_count - no. of other users who are following this user (followers)
        # user.statuses_count - total tweets by user
        # user.created_at - when the user account was created
        # created_at - when the tweet was created
        # retweet_count - no. of retweets
        # (deprecated) user.favourites_count - probably total no. of tweets that is favourited by user
        # retweeted_status.full_text - full text of the tweet
        # tweet.entities['hashtags'] - hashtags in the tweet
# Begin scraping the tweets individually:
        noTweets = 0
        for tweet in tweet_list:
# Pull the values
            username = tweet.user.screen_name
            acctdesc = tweet.user.description
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']
            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text
# Add the 11 variables to the empty list - ith_tweet:
            ith_tweet = [username, acctdesc, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]
# Append to dataframe - db_tweets
            db_tweets.loc[len(db_tweets)] = ith_tweet
# increase counter - noTweets  
            noTweets += 1
        
# Run ended:
    end_run = time.time()
    duration_run = round((end_run-start_run)/60, 2)

    print('no. of tweets scraped for run {} is {}'.format(i + 1, noTweets))
    print('time take for {} run to complete is {} mins'.format(i+1, duration_run))

    time.sleep(0) #15 minute sleep time (920)
# Once all runs have completed, save them to a single csv file:
    from datetime import datetime

    # Obtain timestamp in a readable format
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')
# Define working path and filename
    path = os.getcwd()
    filename = path + to_csv_timestamp + search_words+'_TT.csv'
# Store dataframe in csv with creation date timestamp
    db_tweets.to_csv(filename, index = False)

    program_end = time.time()
    print('Scraping has completed!')
    print('Total time taken to scrap is {} minutes.'.format(round(program_end - program_start)/60, 2))

In [ ]:
# Initialise these variables:
search_words = "#paronacional" 
date_since = "2021-5-05"
numTweets = 1000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 1000
time take for 1 run to complete is 1.08 mins
Scraping has completed!
Total time taken to scrap is 1.0833333333333333 minutes.


In [ ]:
# Initialise these variables:
search_words = "#jamesrodriguez" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 57
time take for 1 run to complete is 0.06 mins
Scraping has completed!
Total time taken to scrap is 0.06666666666666667 minutes.


In [ ]:
# Initialise these variables:
search_words = "#barcelona" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 0
time take for 1 run to complete is 0.01 mins
Scraping has completed!
Total time taken to scrap is 0.0 minutes.


In [ ]:
# Initialise these variables:
search_words = "#seleccioncolombia" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 195
time take for 1 run to complete is 0.17 mins
Scraping has completed!
Total time taken to scrap is 0.16666666666666666 minutes.


In [ ]:
# Initialise these variables:
search_words = "#bitcoin" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 2.35 mins
Scraping has completed!
Total time taken to scrap is 2.35 minutes.


In [ ]:
# Initialise these variables:
search_words = "#ivanduque" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 14.39 mins
Scraping has completed!
Total time taken to scrap is 14.4 minutes.


In [ ]:
# Initialise these variables:
search_words = "#felizviernes" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 15.58 mins
Scraping has completed!
Total time taken to scrap is 15.583333333333334 minutes.


In [ ]:
# Initialise these variables:
search_words = "#felizlunes" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 14.62 mins
Scraping has completed!
Total time taken to scrap is 14.616666666666667 minutes.


In [ ]:
# Initialise these variables:
search_words = "#copalibertadores" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 14.55 mins
Scraping has completed!
Total time taken to scrap is 14.55 minutes.


In [ ]:
# Initialise these variables:
search_words = "#adhoc" 
date_since = "2021-5-19"
numTweets = 1000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 15
time take for 1 run to complete is 0.04 mins
Scraping has completed!
Total time taken to scrap is 0.03333333333333333 minutes.


In [ ]:
# Initialise these variables:
search_words = "#paronacional19M" 
date_since = "2021-5-05"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 2.61 mins
Scraping has completed!
Total time taken to scrap is 2.6166666666666667 minutes.


In [ ]:
# Initialise these variables:
search_words = "juan guillermo cuadrado" 
date_since = "2021-5-19"
numTweets = 1000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 359
time take for 1 run to complete is 0.4 mins
Scraping has completed!
Total time taken to scrap is 0.4 minutes.


In [ ]:
# Initialise these variables:
search_words = "copaamerica" 
date_since = "2021-5-19"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 2.13 mins
Scraping has completed!
Total time taken to scrap is 2.1333333333333333 minutes.


In [ ]:
# Initialise these variables:
search_words = "eurocopa" 
date_since = "2021-5-19"
numTweets = 2000
numRuns = 1
# Call the function scraptweets
scraptweets(search_words, date_since, numTweets, numRuns)

no. of tweets scraped for run 1 is 2000
time take for 1 run to complete is 16.33 mins
Scraping has completed!
Total time taken to scrap is 16.35 minutes.


In [ ]:
paronacional = pd.read_csv('../Twitter_exc20210519_191812#paronacional_TT.csv')
paronacional



,username,acctdesc,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,text,hashtags
0,luisaDavilaSan1,Colombiana\n17 years \nARMY- BLINK,COLOMBIA,120,3,1116,2021-02-24 02:39:26,2021-05-20 00:17:05,1,CON LA POLICÍA Y EL ESMAD NO HAY CASO. NO ESCU...,[]
1,Fotos_Ecuador,Mostrando nuestro hermoso país al mundo.,Ecuador,847,299,6508,2016-09-21 05:49:07,2021-05-20 00:17:04,2,#ParoNacional\nCali\nLa Loma de la Dignidad en...,"[{'text': 'ParoNacional', 'indices': [18, 31]}..."
2,DiegoA3110,❣️. Antiuribista.,NaN,1149,2368,44948,2018-09-02 03:23:38,2021-05-20 00:17:04,2,#ParoNacional\nCali\nLa Loma de la Dignidad en...,"[{'text': 'ParoNacional', 'indices': [18, 31]}..."
3,OscarMonroy,Administrador de Empresas,NaN,1112,244,37939,2009-08-25 18:59:37,2021-05-20 00:17:02,798,#Cali Siguen llenando de vida la Loma de La Di...,"[{'text': 'Cali', 'indices': [20, 25]}, {'text..."
4,rafalatino,"iconoclasta,literato por afición,creativo por ...",Avenida El Cerro Caracas,1551,818,117805,2010-05-08 15:25:05,2021-05-20 00:17:01,5,#19May #Colombia \n#Bogotá Continúan llegando ...,"[{'text': '19May', 'indices': [15, 21]}, {'tex..."
...,...,...,...,...,...,...,...,...,...,...,...
995,BedoyaEsLuis1,Conocimientos en la fe de la razón de la human...,NaN,2589,827,2109,2020-03-31 23:28:58,2021-05-19 23:59:27,11,A propósito de los saqueos y robos hoy en #Bue...,"[{'text': 'Buenventura', 'indices': [61, 73]}]"
996,Ruben0628,"Ing. Administrador. Buen lector, HIncha del ve...",Medellín,818,409,124266,2011-08-06 19:30:40,2021-05-19 23:59:21,6,Bonito gesto hoy en Medellín🥰. Esto es lo que ...,"[{'text': 'ParoNacional19M', 'indices': [109, ..."
997,piedrahitangela,NaN,Colombia,1025,24354,96841,2011-06-29 23:26:47,2021-05-19 23:59:21,385,Cali dice Viva el paro nacional!!\n#19Mayo #Pa...,"[{'text': '19Mayo', 'indices': [48, 55]}, {'te..."
998,colombiamarchas,"Cuenta oficial de Colombia Marcha, medios de c...",NaN,50,14,509,2021-05-13 05:31:48,2021-05-19 23:59:20,11,#ParoNacional #19Mayo #ParoNacionalIndefinido ...,"[{'text': 'ParoNacional', 'indices': [19, 32]}..."


In [ ]:
##GET Twitter DataFrames

In [ ]:
%pwd

'/Users/mac/Documents/Datasets/Twitter_exc'

In [ ]:
paronacional = pd.read_csv('../Twitter_exc20210519_193127#paronacional_TT.csv')
jamesrodriguez = pd.read_csv('../Twitter_exc20210519_193131#jamesrodriguez_TT.csv')
barcelona = pd.read_csv('../Twitter_exc20210519_193132#barcelona_TT.csv')
felizviernes = pd.read_csv('../Twitter_exc20210519_200402#felizviernes_TT.csv')
felizlunes = pd.read_csv('../Twitter_exc20210519_201839#felizlunes_TT.csv')
copalibertadores = pd.read_csv('../Twitter_exc20210519_203312#copalibertadores_TT.csv')
adhoc = pd.read_csv('../Twitter_exc20210519_203314#adhoc_TT.csv')
copaamerica = pd.read_csv('../Twitter_exc20210706_090037copaamerica_TT.csv')
eurocopa = pd.read_csv('../Twitter_exc20210706_091658eurocopa_TT.csv')
bitcoin = pd.read_csv('../Twitter_exc20210519_193403#bitcoin_TT.csv')
ivanduque = pd.read_excel('/Users/mac/Downloads/drive-download-20210702T123910Z-001/DuqueDiariodata.xlsx')

In [ ]:
paronacional_train = paronacional[['text']]
jamesrodriguez_train = jamesrodriguez[['text']]
barcelona_train=barcelona[['text']]
felizviernes_train = felizviernes[['text']]
felizlunes_train = felizlunes[['text']]
copalibertadores_train=copalibertadores[['text']]
adhoc_train = adhoc[['text']]
copaamerica_train = copaamerica[['text']]
eurocopa_train = eurocopa[['text']]
bitcoin_train = bitcoin[['text']]
ivanduque_train = ivanduque[['Tweet']]


In [ ]:
ivanduque_train.rename(columns = {'Tweet':'text'},inplace = True)

/Users/mac/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/pandas/core/frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
ivanduque_train_short = ivanduque_train[:2000]

In [ ]:
paronacional_train['clasificacion'] = 'Política'
jamesrodriguez_train['clasificacion'] = 'Deportes'
barcelona_train['clasificacion'] = 'Deportes'
felizviernes_train['clasificacion'] = 'Otros'
felizlunes_train['clasificacion'] = 'Otros'
copalibertadores_train['clasificacion'] = 'Deportes'
adhoc_train['clasificacion'] = 'Política'
copaamerica_train['clasificacion'] = 'Deportes'
eurocopa_train['clasificacion'] = 'Deportes'
bitcoin_train['clasificacion'] = 'Economía'
ivanduque_train['clasificacion'] = 'Política'

/Users/mac/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/mac/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/mac/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFram

In [ ]:
Consolidado = pd.DataFrame(columns =[['text','Clasificación']])
Consolidado = pd.concat([paronacional_train,
           jamesrodriguez_train,
           barcelona_train,
           felizviernes_train,
           felizlunes_train,
           copalibertadores_train,
           adhoc_train,
           copaamerica_train,
           ivanduque_train_short],
          ignore_index = True)
Consolidado


,text,clasificacion
0,El #ParoNacional ya se tumbó\n1. La reforma tr...,Política
1,Otro logro del #ParoNacional Colombia pierde c...,Política
2,Una ola de protesta vuelve a inundar #Colombia...,Política
3,Una ola de protesta vuelve a inundar #Colombia...,Política
4,Alguien podría decirme que dice la arenga que ...,Política
...,...,...
11069,Día a día los contagios y las muertes siguen a...,Política
11070,"@AForeroM @ClaudiaLopez ""Lo que faltaba, bandi...",Política
11071,RT @juan_alies: La mejor noticia que dió @Ivan...,Política
11072,"@elespectador ""Lo que faltaba, bandidos #Prime...",Política


In [ ]:
##PREPROCCESSING

In [ ]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop_words = stopwords.words('spanish')

import string
alphabet_string = string.ascii_lowercase
#Create a string of all lowercase letters

alphabet_list = list(alphabet_string)
#Create a list of all lowercase letters

two_word_list = []
for a in alphabet_list:
    for i in range(len(alphabet_list)):
        two_word_list.append(a+alphabet_list[i])

[nltk_data] Downloading package stopwords to /Users/mac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
newstopwords = ['x','q','si','ser','d','as','pa','ud','usted','señor','seor']
for i in range(len(two_word_list)):
    newstopwords.append(two_word_list[i])
for element in newstopwords:
    stop_words.append(element)
stop_words

['de',
 'la',
 'que',
 'el',
 'en',
 'y',
 'a',
 'los',
 'del',
 'se',
 'las',
 'por',
 'un',
 'para',
 'con',
 'no',
 'una',
 'su',
 'al',
 'lo',
 'como',
 'más',
 'pero',
 'sus',
 'le',
 'ya',
 'o',
 'este',
 'sí',
 'porque',
 'esta',
 'entre',
 'cuando',
 'muy',
 'sin',
 'sobre',
 'también',
 'me',
 'hasta',
 'hay',
 'donde',
 'quien',
 'desde',
 'todo',
 'nos',
 'durante',
 'todos',
 'uno',
 'les',
 'ni',
 'contra',
 'otros',
 'ese',
 'eso',
 'ante',
 'ellos',
 'e',
 'esto',
 'mí',
 'antes',
 'algunos',
 'qué',
 'unos',
 'yo',
 'otro',
 'otras',
 'otra',
 'él',
 'tanto',
 'esa',
 'estos',
 'mucho',
 'quienes',
 'nada',
 'muchos',
 'cual',
 'poco',
 'ella',
 'estar',
 'estas',
 'algunas',
 'algo',
 'nosotros',
 'mi',
 'mis',
 'tú',
 'te',
 'ti',
 'tu',
 'tus',
 'ellas',
 'nosotras',
 'vosotros',
 'vosotras',
 'os',
 'mío',
 'mía',
 'míos',
 'mías',
 'tuyo',
 'tuya',
 'tuyos',
 'tuyas',
 'suyo',
 'suya',
 'suyos',
 'suyas',
 'nuestro',
 'nuestra',
 'nuestros',
 'nuestras',
 'vuestro'

In [ ]:
def remove_url(txt):
    """Replace URLs found in a text string with nothing 
    (i.e. it will remove the URL from the string).

    Parameters
    ----------
    txt : string
        A text string that you want to parse and remove urls.

    Returns
    -------
    The same txt string with url's removed.
    """

    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())

In [ ]:
def remove_hash_ad_emojis(text):
    return " ".join(re.sub("(RT)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","", text).split())



In [ ]:
Consolidado['text']

0        El #ParoNacional ya se tumbó\n1. La reforma tr...
1        Otro logro del #ParoNacional Colombia pierde c...
2        Una ola de protesta vuelve a inundar #Colombia...
3        Una ola de protesta vuelve a inundar #Colombia...
4        Alguien podría decirme que dice la arenga que ...
                               ...                        
11069    Día a día los contagios y las muertes siguen a...
11070    @AForeroM @ClaudiaLopez "Lo que faltaba, bandi...
11071    RT @juan_alies: La mejor noticia que dió @Ivan...
11072    @elespectador "Lo que faltaba, bandidos #Prime...
11073    RT @AlexLopezMaya: EEUU condiciona recursos a ...
Name: text, Length: 11074, dtype: object

In [ ]:
Consolidado['text']=Consolidado['text'].apply(lambda x: remove_hash_ad_emojis(str(x)))

In [ ]:
from sklearn.model_selection import train_test_split
X = Consolidado['text']
y =Consolidado['clasificacion']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
#tfidf = TfidfVectorizer(stop_words=stop_words)
from sklearn.svm import LinearSVC
clf_pipeline = Pipeline([('tfidf',TfidfVectorizer(stop_words=stop_words)),('clf',LinearSVC())])

In [ ]:
clf_pipeline.fit(X_train,y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(stop_words=['de', 'la', 'que', 'el', 'en', 'y',
                                             'a', 'los', 'del', 'se', 'las',
                                             'por', 'un', 'para', 'con', 'no',
                                             'una', 'su', 'al', 'lo', 'como',
                                             'más', 'pero', 'sus', 'le', 'ya',
                                             'o', 'este', 'sí', 'porque', ...])),
                ('clf', LinearSVC())])

In [ ]:
predictions = clf_pipeline.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score
print(confusion_matrix(y_test,predictions))

[[1284    6   17]
 [   0 1323   16]
 [   3   12  994]]


In [ ]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

    Deportes       1.00      0.98      0.99      1307
       Otros       0.99      0.99      0.99      1339
    Política       0.97      0.99      0.98      1009

    accuracy                           0.99      3655
   macro avg       0.98      0.99      0.98      3655
weighted avg       0.99      0.99      0.99      3655



In [ ]:
print(accuracy_score(y_test,predictions
                    ))

0.9852257181942544


In [ ]:
X_test

5579     Final 1TIndependienteSantaFe est tentado con p...
6779     CopaLibertadores Libertadores RiverPlate Indep...
8350     Me siento muy orgulloso de ser parte de esta s...
7150     Una falta de respeto como le habla a todos los...
7701     Me siento muy orgulloso de ser parte de esta s...
                               ...                        
2175     Con todo el respeto del mundo porque los sbado...
10780    Hoy me quedo con esta frase del Dr comentando ...
2540     BuenosDas y FelizViernes Quedan pocos minutos ...
9711     tengo un pariente que es mamerto y siempre esc...
4993     Hoy puede ser un buen da para que eches a vola...
Name: text, Length: 3655, dtype: object

In [ ]:
clf_pipeline.predict([''])[0]

'Deportes'

In [ ]:
ivanduque_test = ivanduque_train[2000:5000]

In [ ]:
ivanduque_test['prediccion']= ivanduque_test['text'].apply(lambda x:clf_pipeline.predict([x])[0])

/Users/mac/opt/anaconda3/envs/nlp_course/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
accuracy_score(ivanduque_test['clasificacion'],ivanduque_test['prediccion'])

0.9663333333333334

In [ ]:
model = clf_pipeline

In [ ]:
##SAVING MODEL TO PICKLE

In [ ]:
import pickle
# save the model to disk
filename = '/content/twitter_classifier_model_v1.sav'
pickle.dump(model, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

NameError: ignored